# Similaridade de texto (palavras)

A similaridade de texto é utilizada para comparar palavras similares (que foram escritas de forma diferente).
Alguns exemplos:

* `São Paulo`
* `Sao Paulo`
*  `São Paulo   ` (tem espaço no final)
* `são paulo`

Todas as palavras acima se referem a cidade de São Paulo, mas se efetuar umas simples comparação de _strings_ (`str1 == str2`) o resultado será `False`.

É possível tratar esses casos, como (`str.strip()`) para remover espaços no começo/final do texto.
Outro detalhe é `str.lower()` ou `str.upper()` para manter tudo minúsculo/maiúsculo.
Finalmente, é possível remover toda a acentuação para evitar problemas.

Entretanto, essa não é uma boa adordagem (principalmente remover acentuação, as outras duas podem ser boas ideias) visto que estamos alterando o texto para encontrar o nome `São Paulo`.

Uma alternativa é computar um número (entre 0 e 100, por exemplo) que indica o quão similares são duas palavras.
Sendo que, 0 indica palavras completamente diferentes e 100 indica que as palavras são idênticas.
Assim, podemos definir um limiar de forma que, qualquer similaridade acima de 80 aceitamos como algo válido.
O valor de limiar vai variar com o contexto, por isso é importante testar várias abordagens.

Existem várias métricas para calcular essa similaridade: https://en.wikipedia.org/wiki/Edit_distance

Vamos trabalhar com a distância de Levenshtein: https://en.wikipedia.org/wiki/Levenshtein_distance

A distância de Levenshtein calcula o número minímino de alterações (inserir, remover, substituir) que devemos fazer em uma string para que ela fique igual a outra.

Tutorial `fuzzywuzzy` https://www.datacamp.com/community/tutorials/fuzzy-string-python

In [4]:
from fuzzywuzzy import fuzz

/home/julio/Projects/pos-furb-recuperacao/.venv/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
fuzz.ratio('São Paulo', 'Sao Paulo')

89

In [6]:
fuzz.ratio('São Paulo', 'sao paulo')

67

In [7]:
fuzz.ratio('São Paulo', 'São Paulo    ')

82

In [8]:
fuzz.ratio('Sao Paulo', 'São Paulo')

89

Assim, podemos ter uma lista de cidades e ver quais pessoas moram em `São Paulo`

In [10]:
pessoas = [
    {'nome': 'Pessoa A', 'cidade': 'São Paulo'},
    {'nome': 'Pessoa B', 'cidade': ' São Paulo '},
    {'nome': 'Pessoa C', 'cidade': 'Sao Paulo'},
    {'nome': 'Pessoa D', 'cidade': 'sao paulo'},
    {'nome': 'Pessoa E', 'cidade': 'SaoPaulo'},
    {'nome': 'Pessoa F', 'cidade': 'S. Paulo'},
    {'nome': 'Pessoa G', 'cidade': 'Blumenau'},
    {'nome': 'Pessoa H', 'cidade': 'Pomerode'},
    {'nome': 'Pessoa I', 'cidade': 'Rio de Janeiro'},
]

scores = [fuzz.ratio('São Paulo', p['cidade']) for p in pessoas]
scores

[100, 90, 89, 67, 82, 82, 24, 24, 35]

Entretanto, nem sempre é possível comparar o texto diretamente.
Por exemplo, o nome da cidade pode estar no meio de uma frase.
Para isso, é possível fazer uma comparação parcial.

`fuzz.ratio()` compara dois textos por completo, com a similaridade entre eles.

`fuzz.partial_ratio()` faz a comparação em substrings.

In [13]:
fuzz.ratio('São Paulo', 'João mudou-se para São Paulo em 2019')

40

In [11]:
fuzz.partial_ratio('São Paulo', 'João mudou-se para São Paulo em 2019')

100

In [12]:
fuzz.partial_ratio('São Paulo', 'João mudou-se para Sao Paulo em 2019')

89

Outro caso comum, é a ordem dos textos.

Por exemplo, `Comparamos o Dispositivo A com o Dispositivo B` e `Comparamos o Dispositivo B com o Dispositivo A`.

In [14]:
fuzz.partial_ratio('Dispositivo A com Dispositivo B', 'Comparamos Dispositivo A com Dispositivo B')

100

In [17]:
fuzz.partial_ratio('Dispositivo A com Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

55

In [18]:
fuzz.token_sort_ratio('Dispositivo A com Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

85

Outro caso é o uso da comparação `Dispositivo A com Dispositivo B` ou `Dispositivo A x Dispositivo B`.

In [19]:
fuzz.token_sort_ratio('Dispositivo A x Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

76

In [21]:
fuzz.token_set_ratio('Dispositivo A x Dispositivo B', 'Comparamos dispositivo B com dispositivo A')

94

## Processamento de linguagem natural

São técnicas utilizadas para trabalhar com texto que englobam desde _tokenização_ (separar uma string em palavras) até a tradução de texto.

A similaridade de palavras acima demonstra um uso de processamento de linguagem natural.

Nesta parte, vamos ver alguns conceitos básicos utilizando as bibliotecas `NLTK` e `spaCy`.

Uma comparação entre `NLTK` e `spaCy` https://medium.com/@akankshamalhotra24/introduction-to-libraries-of-nlp-in-python-nltk-vs-spacy-42d7b2f128f2

In [8]:
# Fonte http://www.furb.br/web/1704/noticias/furb-esta-entre-4-melhores-de-sc-em-ranking-da-america-latina/7944
texto = '''O Brasil desponta com as melhores Universidades da América Latina, com o total de 3 no top 10,
seguida do Chile, Colômbia e México com 2 em cada país, além da Argentina com uma.
No primeiro lugar está a Pontifícia Universidade Católica do Chile, seguida da Universidade de São Paulo (USP).
'''

## Tokenização

Separara um texto em uma lista de palavras.
Note que a forma mais simples de resolver esse problema é `texto.split(' ')` apenas separando por espaços.
Entretanto, essa abordagem falha em um texto como `... fim do dia.`, onde `dia.` seria um token, mas deveria ser separdo em dois: `dia` e `.`.
O mesmo acontece para nomes, `João B. da Silva`, é importante notar que `B.` não é o final da frase, mas uma abreviação.

In [9]:
from nltk.tokenize import word_tokenize

In [13]:
# Pode ser necessário executar
# import nltk
# nltk.download('punkt')

In [14]:
tokens = word_tokenize(texto)
for (i, token) in enumerate(tokens):
    print(token)
    if i == 10:
        break

O
Brasil
desponta
com
as
melhores
Universidades
da
América
Latina
,


In [15]:
import spacy

In [16]:
# pode ser necessário instalar o modelo
# python -m spacy download pt_core_news_sm
nlp = spacy.load('pt_core_news_sm')

In [17]:
doc = nlp(texto)

In [18]:
for (i, token) in enumerate(doc):
    print(token.text)
    if i == 10:
        break

O
Brasil
desponta
com
as
melhores
Universidades
da
América
Latina
,


In [13]:
# apenas os verbos
verbos = [token.text for token in doc if token.pos_ == 'VERB']
verbos

['desponta', 'top', 'seguida', 'está', 'seguida']

## Remoção de _stopwords_

_Stopwords_ são palavras utilizadas para conectar texto (`de`, `a`, `e`, outros) e não estão ligadas diretamente ao sentido da frase.

In [20]:
from nltk.corpus import stopwords

In [23]:
# pode ser necessário executar
# import nltk
# nltk.download('stopwords')

In [27]:
sws = stopwords.words('portuguese')
sws[:10]

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um']

In [32]:
tokens_sem_stopwords = [t for t in tokens if t not in sws]
print(len(tokens), len(tokens_sem_stopwords))
tokens_sem_stopwords[:10]

60 41


['O',
 'Brasil',
 'desponta',
 'melhores',
 'Universidades',
 'América',
 'Latina',
 ',',
 'total',
 '3']

In [51]:
spacy_tokens_sem_stopwords = [t for t in doc if not t.is_stop]
spacy_tokens_sem_stopwords[:10]

[O, Brasil, desponta, melhores, Universidades, América, Latina, ,, o, total]

## Stem

_Stem_ é um método para reduzir uma palavra ao radical/raiz.
Como os verbos são conjugados, pode ser difícil trabalhar com todas as variações.
Uma forma de reduzir esse problema, é trabalhar apenas com o radical do verbo.
Por: `copiar` -> `copi`

In [35]:
# pode ser necessário executar
# import nltk
# nltk.download('rslp')

In [36]:
stemmer = nltk.stem.RSLPStemmer()

In [44]:
stemmer.stem("necessário")

'necess'

In [46]:
stemmer.stem("necessidade")

'necess'

In [45]:
stemmer.stem("desnecessário")

'desnecess'

In [43]:
tokens_stem = [stemmer.stem(t) for t in tokens_sem_stopwords]
tokens_stem[:10]

['o',
 'brasil',
 'despont',
 'melhor',
 'univers',
 'amér',
 'latin',
 ',',
 'total',
 '3']

## Lemmatization

Enquanto _stemming_ reduz as palavras removendo partes no começo e final, _lemmatization_ reduz a palavra na forma do dicionário.

Para mais informações: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html

In [54]:
lemmas = [t.lemma_ for t in spacy_tokens_sem_stopwords]
lemmas[:10]

['O',
 'Brasil',
 'despontar',
 'melhorar',
 'Universidades',
 'América',
 'Latina',
 ',',
 'o',
 'total']

## Part of Speech (POS) tagging

Anota as palavras de acordo com a sua função linguística na frase.
Ou seja, indica se uma palavra é um verbo, substantivo, pontuação, outros.

In [55]:
lemmas = [t.pos_ for t in spacy_tokens_sem_stopwords]
lemmas[:10]

['DET',
 'PROPN',
 'VERB',
 'ADJ',
 'PROPN',
 'PROPN',
 'PROPN',
 'PUNCT',
 'DET',
 'NOUN']

In [56]:
# pegando apenas os verbos
for token in filter(lambda x: x.pos_ == 'VERB', doc):
    print(token)

desponta
top
seguida
está
seguida


In [57]:
# pegando apenas os substantivos
for token in filter(lambda x: x.pos_ == 'NOUN', doc):
    print(token)

total
país
lugar
Pontifícia


In [59]:
# pegando apenas os nomes próprios
for token in filter(lambda x: x.pos_ == 'PROPN', doc):
    print(token)

Brasil
Universidades
América
Latina
10
do
Chile
Colômbia
México
Argentina
Universidade
Católica
Chile
da
Universidade
São
Paulo
USP


## Named Entity Recognition (NER)

Reconhece e classifica entidades no texto

In [61]:
for entity in doc.ents:
    print(entity.text, entity.label_)

Brasil LOC
Universidades da América Latina PER
Chile LOC
Colômbia LOC
México LOC
Argentina LOC
Pontifícia Universidade Católica do Chile LOC
Universidade de São Paulo LOC
USP LOC
